In [ ]:
from mitigate_disparity import mitigate_disparity

est = mitigate_disparity(
    dataset='data/mimic/development_dataset.csv',
    protected_features=[
        'ethnicity_0',
        'ethnicity_1',
        'ethnicity_2',
        'ethnicity_3',
        'ethnicity_4',
        'insurance_0',
        'insurance_1',
        'insurance_2'
    ]
)